In [ ]:
# !pip install opencv-python scikit-learn scikit-image matplotlib spectral keras_tuner vis
# !pip install tensorflow numpy pandas
# !pip install spectral

In [8]:
import tensorflow as tf

In [9]:
from tensorflow import keras as keras
from keras import layers as layers

In [32]:
import os, timeit
from skimage.filters import threshold_otsu
import numpy as np
from math import inf as inf

In [11]:
from spectral.io import envi as envi
from spectral import imshow

In [12]:
from sklearn.decomposition import IncrementalPCA

In [13]:
import sys

In [14]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [15]:
from sys import platform
DATA_DIRECTORY = ""
SLASH = ""
if platform == "linux" or platform == "linux2":
    DATA_DIRECTORY = "/home/nitintyagi/wheat data/BULK/"
    SLASH = "/"
elif platform == "win32":
    DATA_DIRECTORY = "D:\mvl\wheat\data\BULK\\"
    SLASH="\\"

In [16]:
#Testing Constants
TESTING = False

#Constants
BAND_NUMBER = 60
FILLED_AREA_RATIO = 0.40
IMAGE_COUNT = int(8/4)
NUM_VARIETIES = 2
NUM_OF_BANDS = 15
FIRST_BAND = 21
LAST_BAND = 149

IMAGE_WIDTH = 30
IMAGE_HEIGHT = 30

ACTIVATION_TYPE =  "relu"
BATCH_SIZE = 2*NUM_VARIETIES

LEARNING_RATE_BASE = 0.0001

In [17]:
def start_timer():
    print("Testing started")
    return timeit.default_timer()

def end_timer():
    return timeit.default_timer()

def show_time(tic,toc): 
    test_time = toc - tic
    print('Testing time (s) = ' + str(test_time) + '\n')

In [18]:
def exactPathHDR(variety,file):
    return DATA_DIRECTORY+variety+SLASH+file+".bil.hdr"

def exactPathBIL(variety,file):
    return DATA_DIRECTORY+variety+SLASH+file+".bil"

In [19]:
def getROI(img,band_number):
    img_band = img.read_band(band_number)
    threshold = threshold_otsu(img_band)
    roi=[]
    for x in range(img_band.shape[0]):
        a=[]
        for y in range(img_band.shape[1]):
            if img_band[x][y]>threshold:
                a.append(1)
            else:
                a.append(0)
        roi.append(a)
    return roi

In [20]:
#Returns range for x and y from where we have to crop images
def getRangeXandY(img,band_number):
    img_band = img.read_band(band_number)
    roi = getROI(img,band_number)
    xmin = inf
    xmax = 0
    ymin = inf
    ymax = 0
    for x in range(img_band.shape[0]):
        for y in range(img_band.shape[1]):
            if roi[x][y]==1:
                if x<xmin:
                    xmin=x
                if x>xmax:
                    xmax=x
                if y<ymin:
                    ymin=y
                if y>ymax:
                    ymax=y
    return xmin, xmax, ymin, ymax

In [21]:
def getCroppedImage(img,band_number):
    xmin, xmax, ymin, ymax = getRangeXandY(img,band_number)
    new_img = img[xmin:xmax, ymin:ymax, :]
    return new_img    

In [22]:
def getCroppedROI(img,band_number):
    xmin, xmax, ymin, ymax = getRangeXandY(img,band_number)
    roi = np.array(getROI(img,band_number))
    roi = roi[xmin:xmax, ymin:ymax]
    return roi   

In [23]:
def getUsefulImage(img,band_number):
    crop_img = getCroppedImage(img,band_number)
    crop_roi = getCroppedROI(img,band_number)
    for x in range(crop_img.shape[2]):
        band = crop_img[:,:,x]
        crop_img[:,:,x] = band*crop_roi
    return crop_img

In [24]:
data_augmentation = keras.Sequential([
    layers.RandomCrop(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
    layers.RandomRotation(factor=(-0.1, 0.1)),
    layers.RandomZoom(height_factor=(-0.1, 0.1), width_factor=(-0.1,0.1)),
    layers.RandomFlip(mode="horizontal_and_vertical", seed=None)
])

def getAugumentedImage(img,band_number):
    new_img = getUsefulImage(img,band_number)
    augmented_image = data_augmentation(new_img) 
    return augmented_image

def checkAugumentedImage(augmented_image):
    aug_band = augmented_image[:,:,0]
    filled_area_ratio = (np.count_nonzero(aug_band))/(aug_band.shape[0]*aug_band.shape[1])
    if filled_area_ratio > FILLED_AREA_RATIO :
        return True
    else:
        return False

In [25]:
## Dimensional Reduction Method
def DL_Method(HSI, numComponents = NUM_OF_BANDS):
    RHSI = np.reshape(HSI, (-1, HSI.shape[2]))
    n_batches = 10
    inc_pca = IncrementalPCA(n_components=numComponents)
    for X_batch in np.array_split(RHSI, n_batches):
        inc_pca.partial_fit(X_batch)
    X_ipca = inc_pca.transform(RHSI)
    RHSI = np.reshape(X_ipca, (HSI.shape[0],HSI.shape[1], numComponents))
    return RHSI

In [26]:
# List for All varieties
VARIETIES = []
VARIETIES_CODE = {}

for name in os.listdir(DATA_DIRECTORY):
    if (name.endswith(".hdr") or name.endswith(".bil")):
        continue
    VARIETIES_CODE[name] = len(VARIETIES)
    VARIETIES.append(name)
    if len(VARIETIES)==NUM_VARIETIES:
        break

In [27]:
#List for all file names in varities
FILES = []
MAX_FILE_NUM = 4
for x in range(1,MAX_FILE_NUM+1):
    FILES.append("B_"+str(x))

In [28]:
#List of all images
images = []
images_label = []
for v in VARIETIES:
    for f in FILES:
        try:
            img = envi.open(exactPathHDR(v,f),exactPathBIL(v,f))
            images.append(img)
            images_label.append(v)
        except:
            pass

In [29]:
train_dataset = []
train_dataset_label = []
test_dataset = []
test_dataset_label = []

In [30]:
i = 0

In [33]:
tic = start_timer()
for index, img in enumerate(images):
    count = 0
    label = images_label[index]
    while count<IMAGE_COUNT:
        aug_img = getAugumentedImage(img,BAND_NUMBER)
        
        if checkAugumentedImage(aug_img):
            aug_img = DL_Method(aug_img[:,:,FIRST_BAND:LAST_BAND+1])
            if count%5 == 0:
                test_dataset.append(aug_img)
                test_dataset_label.append(label)
            else:
                train_dataset.append(aug_img)
                train_dataset_label.append(label)
            count+=1 
        
        i+=1
        print(i,count)    
    
    if TESTING:
        break
        
toc = end_timer()
show_time(tic,toc)

Testing started
1 0
2 1
3 2
4 1
5 1
6 1
7 1
8 2
9 1
10 2
11 1


C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\decomposition\_incremental_pca.py:337: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio = S**2 / np.sum(col_var * n_total_samples)
C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\decomposition\_incremental_pca.py:337: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio = S**2 / np.sum(col_var * n_total_samples)


12 2
13 1
14 2
15 1
16 2
17 1
18 2
19 1
20 2
Testing time (s) = 97.95025549999991



In [34]:
train_dataset = np.array(train_dataset)
train_dataset_label = np.array([VARIETIES_CODE[label] for label in train_dataset_label])
test_dataset = np.array(test_dataset)
test_dataset_label = np.array([VARIETIES_CODE[label] for label in test_dataset_label])

In [35]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model
from keras.layers import Input, Conv2D, MaxPool2D, MaxPooling2D, Activation, Flatten, Dense, AveragePooling2D, GlobalAveragePooling2D
from keras.layers.core import Dropout
from keras.optimizers import Adam

In [36]:
def normalizeDataWholeSeed(data,normalization_type='max'):
    
    if normalization_type == 'max':
        for idx in range(data.shape[0]):
            data[idx,:,:,:] = data[idx,:,:,:]/np.max(abs(data[idx,:,:,:]))
            
    elif normalization_type == 'l2norm':
        from numpy import linalg as LA
        for idx in range(data.shape[0]):
            data[idx,:,:,:] = data[idx,:,:,:]/LA.norm(data[idx,:,:,:])       
        
    return data

In [37]:
def inception(x,filters_1x1,filters_3x3_reduce,filters_3x3,filters_5x5_reduce,filters_5x5,filters_pool,activation_type='relu'):
    path1 = Conv2D(filters_1x1,        (1, 1), padding='same', activation=activation_type)(x)
    
    path2 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation=activation_type)(x)
    path2 = Conv2D(filters_3x3,        (1, 1), padding='same', activation=activation_type)(path2)
    
    path3 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation=activation_type)(x)
    path3 = Conv2D(filters_5x5,        (1, 1), padding='same', activation=activation_type)(path3)
    
    path4 = MaxPool2D((3, 3),  strides=(1, 1), padding='same')(x)
    path4 = Conv2D(filters_pool,       (1, 1), padding='same', activation=activation_type)(path4)
    
    return tf.concat([path1, path2, path3, path4], axis=3)

In [63]:
def auxiliary_classifier(x,num_classes,activation_type='relu'):
    aux = AveragePooling2D((5, 5), strides=3)(x)
    aux = Conv2D(128, 1, padding='same', activation=activation_type)(aux)
    aux = Flatten()(aux)
    aux = Dense(1024, activation=activation_type)(aux)
    aux = Dropout(0.7)(aux)
    aux = Dense(num_classes, activation='softmax')(aux)
    return aux

In [64]:
def GoogleNetModel(data_num_rows, data_num_cols, num_input_chans=1, num_classes=NUM_VARIETIES, activation_type='relu', dropout_rate=0.0):

    inp = Input(shape=(data_num_rows, data_num_cols, num_input_chans))
    input_tensor = layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear", input_shape=train_dataset.shape[1:])(inp)
    x = Conv2D(64,  7, strides=2, padding='same', activation=activation_type)(input_tensor)
    x = MaxPooling2D(3, strides=2)(x)
    x = Conv2D(64,  1, strides=1, padding='same', activation=activation_type)(x)
    x = Conv2D(192, 3, strides=1, padding='same', activation=activation_type)(x)
    x = MaxPooling2D(3, strides=2)(x)
    x = inception(x, filters_1x1=64 , filters_3x3_reduce=96 , filters_3x3=128, filters_5x5_reduce=16, filters_5x5=32, filters_pool=32)
    x = inception(x, filters_1x1=128, filters_3x3_reduce=128, filters_3x3=192, filters_5x5_reduce=32, filters_5x5=96, filters_pool=64)
    x = MaxPooling2D(3, strides=2)(x)
    x = inception(x, filters_1x1=192, filters_3x3_reduce=96 , filters_3x3=208, filters_5x5_reduce=16, filters_5x5=48, filters_pool=64)
    
    aux1 = auxiliary_classifier(x,num_classes)
    
    x = inception(x, filters_1x1=160, filters_3x3_reduce=112, filters_3x3=224, filters_5x5_reduce=24, filters_5x5=64, filters_pool=64)
    x = inception(x, filters_1x1=128, filters_3x3_reduce=128, filters_3x3=256, filters_5x5_reduce=24, filters_5x5=64, filters_pool=64)
    x = inception(x, filters_1x1=112, filters_3x3_reduce=144, filters_3x3=288, filters_5x5_reduce=32, filters_5x5=64, filters_pool=64)
    
    aux2 = auxiliary_classifier(x,num_classes)
    
    x = inception(x, filters_1x1=256, filters_3x3_reduce=160, filters_3x3=320, filters_5x5_reduce=32, filters_5x5=128, filters_pool=128)
    x = MaxPooling2D(3, strides=2)(x)
    x = inception(x, filters_1x1=256, filters_3x3_reduce=160, filters_3x3=320, filters_5x5_reduce=32, filters_5x5=128, filters_pool=128)
    x = inception(x, filters_1x1=384, filters_3x3_reduce=192, filters_3x3=384, filters_5x5_reduce=48, filters_5x5=128, filters_pool=128)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate)(x)
    out = Dense(num_classes, activation='softmax')(x)
    
    return Model(inputs = inp, outputs = [out, aux1, aux2])

In [65]:
def getGoogleNetModel():
    learning_rate_base = LEARNING_RATE_BASE
    activation_type = ACTIVATION_TYPE
    wheat_types =  VARIETIES
    num_classes = len(wheat_types)
    dropout_rate = 0.4
    print("--------------Load Data--------------")

    x_training = np.array(train_dataset)
    labels_training = np.array(train_dataset_label)
    
    # Normalize the data
    x_training = normalizeDataWholeSeed(x_training)
    
    # Extract some information
    num_train = x_training.shape[0]
    N_spatial = x_training.shape[1:3]
    N_channel = x_training.shape[3]
    
    print("--------------Done--------------")
    
    ############ Create a model ############
    print("--------------Create a model--------------")
    
    # Generate a model
    model = GoogleNetModel(data_num_rows = N_spatial[0], 
                           data_num_cols = N_spatial[1],
                           num_input_chans = N_channel, 
                           num_classes = num_classes,
                           activation_type = activation_type,
                           dropout_rate = dropout_rate)

    # Compile the model
    adam_opt = Adam(learning_rate=LEARNING_RATE_BASE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)
    model.compile(optimizer=adam_opt, loss=[losses.sparse_categorical_crossentropy,losses.sparse_categorical_crossentropy,losses.sparse_categorical_crossentropy],loss_weights=[1, 0.3, 0.3],metrics=['accuracy'])
    print("---------Completed---------")
    return model

In [77]:
model = getGoogleNetModel()

--------------Load Data--------------
--------------Done--------------
--------------Create a model--------------
---------Completed---------


In [69]:
x_train = []
y_train = []
x_val = []
y_val = []

for i in range(len(train_dataset)):
    if i%5==0:
        x_val.append(train_dataset[i])
        y_val.append(train_dataset_label[i])
    else:
        x_train.append(train_dataset[i])
        y_train.append(train_dataset_label[i])
        
x_train = np.array(x_train)
y_train = np.array(y_train)
y_train = [y_train,y_train,y_train]

x_val = np.array(x_val)
y_val = np.array(y_val)
y_val = [y_val,y_val,y_val]

In [79]:
for x in range(2):
    print("From epochs: ",20*x+1," to ",20+20*x)
    tic = start_timer()
    model.fit(x_train ,y_train ,batch_size=BATCH_SIZE ,epochs=20, verbose=2, validation_data=(x_val, y_val), shuffle=True)
    toc = end_timer()
    show_time(tic,toc)
    print("for testing")
    print(model.evaluate(test_dataset,test_dataset_label))
    print("for training")
    print(model.evaluate(train_dataset,train_dataset_label))

From epochs:  1  to  20
Testing started
Epoch 1/20
2/2 - 1s - loss: 0.7284 - dense_9_loss: 0.4145 - dense_6_loss: 0.4963 - dense_8_loss: 0.5501 - dense_9_accuracy: 1.0000 - dense_6_accuracy: 1.0000 - dense_8_accuracy: 0.5000 - val_loss: 0.5902 - val_dense_9_loss: 0.3032 - val_dense_6_loss: 0.5086 - val_dense_8_loss: 0.4480 - val_dense_9_accuracy: 1.0000 - val_dense_6_accuracy: 1.0000 - val_dense_8_accuracy: 1.0000 - 909ms/epoch - 455ms/step
Epoch 2/20
2/2 - 1s - loss: 0.5911 - dense_9_loss: 0.3118 - dense_6_loss: 0.4512 - dense_8_loss: 0.4799 - dense_9_accuracy: 1.0000 - dense_6_accuracy: 1.0000 - dense_8_accuracy: 1.0000 - val_loss: 0.5615 - val_dense_9_loss: 0.2841 - val_dense_6_loss: 0.4933 - val_dense_8_loss: 0.4313 - val_dense_9_accuracy: 1.0000 - val_dense_6_accuracy: 1.0000 - val_dense_8_accuracy: 1.0000 - 929ms/epoch - 465ms/step
Epoch 3/20
2/2 - 1s - loss: 0.6416 - dense_9_loss: 0.3452 - dense_6_loss: 0.5064 - dense_8_loss: 0.4816 - dense_9_accuracy: 1.0000 - dense_6_accuracy:

1/1 [==============================] - 0s 270ms/step - loss: 2.8445 - dense_9_loss: 2.2033 - dense_6_loss: 0.6740 - dense_8_loss: 1.4633 - dense_9_accuracy: 0.6250 - dense_6_accuracy: 0.6250 - dense_8_accuracy: 0.6250
[2.8444600105285645, 2.203284740447998, 0.6739528179168701, 1.4632978439331055, 0.625, 0.625, 0.625]
for training
1/1 [==============================] - 0s 266ms/step - loss: 0.0674 - dense_9_loss: 0.0040 - dense_6_loss: 0.1933 - dense_8_loss: 0.0179 - dense_9_accuracy: 1.0000 - dense_6_accuracy: 1.0000 - dense_8_accuracy: 1.0000
[0.06738456338644028, 0.004039854276925325, 0.1932956725358963, 0.017853353172540665, 1.0, 1.0, 1.0]
From epochs:  21  to  40
Testing started
Epoch 1/20
2/2 - 1s - loss: 0.1136 - dense_9_loss: 0.0067 - dense_6_loss: 0.2388 - dense_8_loss: 0.1177 - dense_9_accuracy: 1.0000 - dense_6_accuracy: 1.0000 - dense_8_accuracy: 1.0000 - val_loss: 0.0453 - val_dense_9_loss: 8.4784e-04 - val_dense_6_loss: 0.1414 - val_dense_8_loss: 0.0067 - val_dense_9_accur

Epoch 20/20
2/2 - 1s - loss: 0.0281 - dense_9_loss: 0.0014 - dense_6_loss: 0.0799 - dense_8_loss: 0.0091 - dense_9_accuracy: 1.0000 - dense_6_accuracy: 1.0000 - dense_8_accuracy: 1.0000 - val_loss: 0.0108 - val_dense_9_loss: 6.7887e-05 - val_dense_6_loss: 0.0350 - val_dense_8_loss: 6.4106e-04 - val_dense_9_accuracy: 1.0000 - val_dense_6_accuracy: 1.0000 - val_dense_8_accuracy: 1.0000 - 860ms/epoch - 430ms/step
Testing time (s) = 18.860364699998172

for testing
1/1 [==============================] - 0s 255ms/step - loss: 4.1830 - dense_9_loss: 3.1663 - dense_6_loss: 1.1270 - dense_8_loss: 2.2617 - dense_9_accuracy: 0.6250 - dense_6_accuracy: 0.6250 - dense_8_accuracy: 0.6250
[4.182954788208008, 3.166342258453369, 1.1270017623901367, 2.261706829071045, 0.625, 0.625, 0.625]
for training
1/1 [==============================] - 0s 271ms/step - loss: 0.0215 - dense_9_loss: 8.9080e-04 - dense_6_loss: 0.0658 - dense_8_loss: 0.0029 - dense_9_accuracy: 1.0000 - dense_6_accuracy: 1.0000 - dense_8_

In [74]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 30, 30, 15)  0           []                               
                                ]                                                                 
                                                                                                  
 resizing_2 (Resizing)          (None, 224, 224, 15  0           ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_42 (Conv2D)             (None, 112, 112, 64  47104       ['resizing_2[0][0]']             
                                )                                                             

                                                                  'conv2d_62[0][0]']              
                                                                                                  
 conv2d_65 (Conv2D)             (None, 13, 13, 112)  57456       ['tf.concat_8[0][0]']            
                                                                                                  
 conv2d_67 (Conv2D)             (None, 13, 13, 24)   12312       ['tf.concat_8[0][0]']            
                                                                                                  
 max_pooling2d_18 (MaxPooling2D  (None, 13, 13, 512)  0          ['tf.concat_8[0][0]']            
 )                                                                                                
                                                                                                  
 conv2d_64 (Conv2D)             (None, 13, 13, 160)  82080       ['tf.concat_8[0][0]']            
          

 )                                                                                                
                                                                                                  
 conv2d_90 (Conv2D)             (None, 6, 6, 160)    133280      ['max_pooling2d_22[0][0]']       
                                                                                                  
 conv2d_92 (Conv2D)             (None, 6, 6, 32)     26656       ['max_pooling2d_22[0][0]']       
                                                                                                  
 max_pooling2d_23 (MaxPooling2D  (None, 6, 6, 832)   0           ['max_pooling2d_22[0][0]']       
 )                                                                                                
                                                                                                  
 conv2d_89 (Conv2D)             (None, 6, 6, 256)    213248      ['max_pooling2d_22[0][0]']       
          